In [8]:
import django
import pandas as pd
from django.template import loader, Context
import mysql.connector
from jinja2 import Environment, FileSystemLoader
import pdfkit
import os

class dbResolveContas:
    
    def __init__(self):
            
        self.host     = '35.237.180.90'
        self.database = 'prd-resolvecontas'
        self.user     = 'prd_user_powerbi'
        self.password = '("rYLs+nER0i&Trc'
                
        
    def conecta_db(self):
        conn = mysql.connector.connect(
                host     = self.host,
                database = self.database,
                user     = self.user,
                password = self.password
            )
        return conn
    
    
    def retornaAcordos(self):
        try:
            conn = self.conecta_db()
            QUERY_PARCELAS = f"""
                                SELECT * FROM acordo a          
                                INNER JOIN contrato c ON a.contrato_id = c.id
                                INNER JOIN devedor d ON c.devedor_id = d.id
                                WHERE a.dataCriacao = '2025-04-25' AND closer_id = 58
                                AND created_by = 70 AND active = 'ATIVO'
                            """
        except Exception as e:
            return None
        acordos = pd.read_sql(QUERY_PARCELAS, conn)
        return acordos

In [9]:
resolve = dbResolveContas()
acordos = resolve.retornaAcordos()

In [10]:
# Dicionários que você usava na view
dict_empresa = {
    1: 'Acrux Securitizadora SA',
    2: 'Acrux Securitizadora SA',
    3: 'Betacrux Securitizadora LTDA',
    4: 'Acrux Securitizadora SA',
    5: 'Acrux Securitizadora SA',
}

dict_produto = {
    1: 'Empréstimo Pessoal',
    2: 'Empréstimo Consignado',
    3: 'Empréstimo Pessoal',
    4: 'Empréstimo Pessoal',
    5: 'Empréstimo Pessoal'
}

dict_credor = {
    1: 'Banco BMG',
    2: 'Banco BMG',
    3: 'Banco BMG',
    4: 'Open Co/Geru',
    5: 'Open Co/Rebel'
}

dict_cessao = {
    1: '30/12/2020',
    2: '30/12/2021',
    3: '29/06/2022',
    4: '28/03/2025',
    5: '28/03/2025'
}

In [11]:
import os
import sys
import django

# 1. Aponta para o diretório onde está a PASTA servicesweb
sys.path.append('/home/anderson/dev/django/acrux/sa-parceiros')

# 2. Define o módulo settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'servicesweb.settings')

# 3. Inicializa Django
django.setup()

# Agora testando se carregou:
from django.conf import settings

print('DEBUG:', settings.DEBUG)
print('ROOT_URLCONF:', getattr(settings, 'ROOT_URLCONF', None))
print('BASE_DIR:', getattr(settings, 'BASE_DIR', None))


DEBUG: True
ROOT_URLCONF: servicesweb.urls
BASE_DIR: /home/anderson/dev/django/acrux/sa-parceiros


In [5]:
template = loader.get_template('t_proposta.html')

In [13]:
for _, row in acordos[:2].iterrows():
    acordo = {
        'nome': row['nome'],
        'valor_pendente': row['valor'],
        'valor_do_acordo': row['valor'],
        'nome_credor_atual': dict_empresa.get(row['carteira_id'], ''),
        'nome_credor_original': dict_credor.get(row['carteira_id'], ''),
        'tipo_produto': dict_produto.get(row['carteira_id'], ''),
        'data_cessao': dict_cessao.get(row['carteira_id'], ''),
        'valor_desconto': row['valor'] - row['valor'],
    }
    # Configurações
    options = {
        'encoding': "UTF-8",
    }


    
    context = {'acordo': acordo, 'falha': False}
    html_content = template.render(context)

    # Caminho absoluto para a pasta onde estão os seus arquivos estáticos
    base_path = 'file://' + os.path.abspath('static')

    output_path = f'pdfs/proposta_{row["id"]}.pdf'

    pdfkit.from_string(html_content, output_path, options=options)
    print(f"PDF gerado: {output_path}")

OSError: wkhtmltopdf reported an error:
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Exit with code 1 due to network error: ContentNotFoundError


In [7]:
!pip install pdfkit



[notice] A new release of pip available: 22.2.2 -> 25.1
[notice] To update, run: pip install --upgrade pip
